# Lahore Campus

In [55]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import re

In [56]:
URL = "http://lhr.nu.edu.pk/faculty/"
r = requests.get(URL) 
print(r) 

<Response [200]>


In [57]:
#print(r.content)
soup = BeautifulSoup(r.content, 'html.parser') 

In [58]:
f = soup.findAll('div', class_='facultyCard')

In [59]:
def parse_profile(URL_):
    r1 = requests.get(URL_) 
    soup1 = BeautifulSoup(r1.content, 'html.parser') 
    f1 = soup1.find('div', class_='facultyCard')
    ext=f1.find('p',class_='mt-0 text-center').text.split(':')[-1]
    #try:
       # text=soup1.find('p',class_='text-justify').text
       # pattern = r"(?:Department|Faculty) of ([\w\s&]+)"
       # matches = re.findall(pattern, text)
       # if matches:
       #     department = matches[0]
       # else:
          #  department='Not Mentioned'
    #except:
       # department='Not Mentioned'
    soup2=soup1.find('div',class_='col-lg-8 col-md-6 col-sm-12 text-justify')
    edu=soup2.find('ul').text.split('\n')[1]
    pattern = r',|\('
    edu=re.split(pattern,edu)[0]
    return ext,edu

# Getting FSC emails for dept's verification

In [60]:
cs_emails=[]
ds_emails=[]
se_emails=[]
r1_fsc = requests.get("https://lhr.nu.edu.pk/fsc/faculty/") 
soup1_fsc = BeautifulSoup(r1_fsc.content, 'html.parser')
fsc=soup1_fsc.findAll('div',{ "id" : "faculty-tab" })
cs=fsc[0]
ds=fsc[1]
se=fsc[2]
cs_cards=cs.findAll('div',class_='facultyCard')
for i in range(len(cs_cards)):
    cs_cd=cs_cards[i]
    cs_emails.append(cs_cd.find('p',class_='mb-0 text-center').text)
ds_cards=ds.findAll('div',class_='facultyCard')
for i in range(len(ds_cards)):
    ds_cd=ds_cards[i]
    ds_emails.append(ds_cd.find('p',class_='mb-0 text-center').text)
se_cards=se.findAll('div',class_='facultyCard')
for i in range(len(se_cards)):
    se_cd=se_cards[i]
    se_emails.append(se_cd.find('p',class_='mb-0 text-center').text)

    

# Getting other dept's emails

In [61]:
ee_emails=[]
cv_emails=[]
fsm_emails=[]
ss_emails=[]

r1_oth = requests.get("https://lhr.nu.edu.pk/faculty/") 
soup1_oth = BeautifulSoup(r1_oth.content, 'html.parser')
oth=soup1_oth.findAll('div',{ "id" : "faculty-tab" })
ee=oth[1]
cv=oth[2]
fsm=oth[3]
ss=oth[4]
ee_cards=ee.findAll('div',class_='facultyCard')
cv_cards=cv.findAll('div',class_='facultyCard')
fsm_cards=fsm.findAll('div',class_='facultyCard')
ss_cards=ss.findAll('div',class_='facultyCard')

for i in range(len(ee_cards)):
    ee_cd=ee_cards[i]
    ee_emails.append(ee_cd.find('p',class_='mb-0 text-center').text)

for i in range(len(cv_cards)):
    cv_cd=cv_cards[i]
    cv_emails.append(cv_cd.find('p',class_='mb-0 text-center').text)

for i in range(len(fsm_cards)):
    fsm_cd=fsm_cards[i]
    fsm_emails.append(fsm_cd.find('p',class_='mb-0 text-center').text)

for i in range(len(ss_cards)):
    ss_cd=ss_cards[i]
    ss_emails.append(ss_cd.find('p',class_='mb-0 text-center').text)



In [62]:
def get_dept_lhr(email):
    if(email in cs_emails ):
        return "CS"
    elif(email in ds_emails):
        return "DS"
    elif(email in se_emails):
        return "SE"
    elif(email in ee_emails):
        return "EE"
    elif(email in cv_emails):
        return "CE"
    elif(email in fsm_emails):
        return "FSM"
    elif(email in ss_emails):
        return "S&H"
    else :
        return "Not Mentioned"

In [63]:
id_=[]
name=[]
email=[]
desig=[]
img=[]
profile=[]
HEC=[]
Ext=[]
HighestEd=[]
Dept=[]
for i in range(len(f)):
    faculty = f[i]
    try:
        id_.append(int(faculty.find('a',class_='faculty-link')['href'].split('/')[-1]))
    except:
        id_.append(-1)
    name.append(faculty.find('h5', class_='text-center').text)
    desig.append(faculty.find('p',class_='small').text.strip().split('\n')[0])
    email.append(faculty.find('p', class_='mb-0').text)
    img.append('https://lhr.nu.edu.pk'+faculty.find('img', class_='card-img-top')['src'])
    temp_url='https://lhr.nu.edu.pk'+faculty.find('a',class_='faculty-link')['href']
    #if id[-1]==-1:
        #profile.append(temp_url)
        #HEC.append(-1)
    profile.append(temp_url)
    Extension,HighestEdu=parse_profile(temp_url)
    #print(Extension,HighestEdu,dep)
    Ext.append(Extension)
    HighestEd.append(HighestEdu)
    Dept.append(get_dept_lhr(email[-1]))
    hec_approved_text= faculty.find('p', class_='small').text
    HEC.append("HEC Approved" in hec_approved_text)

In [61]:
print(len(id_))
print(len(name))
print(len(email))
print(len(desig))
print(len(img))
print(len(profile))
print(len(HEC))
print(len(Ext))
print(len(HighestEd))
print(len(Dept))

184
184
184
184
184
184
184
184
184
184


In [62]:
df1= pd.DataFrame({'ID':id_,
                  'Name':name,
                  'Designation':desig,
                   'HEC':HEC,
                   'Highest':HighestEd,
                   'Email':email,
                   'Department':Dept,
                   'Extension':Ext,
                   'Campus':'Lhr',
                   'Image':img,
                  'Profile':profile
                  })

In [67]:
#pd.set_option('display.max_columns', None)
#print(df)
df1['Extension'] = df1['Extension'].replace('None',-1)
df1['Extension'] = df1['Extension'].astype(int)
df1.to_csv('Lhr_FACULTY.csv')
df1.dtypes

ID              int64
Name           object
Designation    object
HEC              bool
Highest        object
Email          object
Department     object
Extension       int32
Campus         object
Image          object
Profile        object
dtype: object

# ISB Campus

In [68]:
#URL_isb = "http://isb.nu.edu.pk/Faculty/allfaculty/"
#r_isb= requests.get(URL_isb)
#soup_isb=BeautifulSoup(r_isb.content, 'html.parser') 
#f_isb = soup_isb.findAll('div', class_='our-team')


In [103]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [69]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService

webdriver_path = 'chromedriver.exe'
options = webdriver.ChromeOptions()
options.headless = True
service = ChromeService(executable_path=webdriver_path)
driver = webdriver.Chrome(service=service, options=options)

#URL_isb = "http://isb.nu.edu.pk/Faculty/allfaculty/"
#driver.get(URL_isb)

#driver.implicitly_wait(8) 

#f_isb= driver.page_source
#soup = BeautifulSoup(f_isb, "html.parser")

#driver.quit()


C:\Users\PC\AppData\Local\Temp\ipykernel_11112\1600785134.py:7: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [70]:
def parse_profile_isb(URL_isb):
    driver.get(URL_isb)
    driver.implicitly_wait(9)
    f_isb= driver.page_source
    soup1= BeautifulSoup(f_isb, "html.parser")
    element = driver.find_element(By.ID,"DegreeName")
    hEdu= element.find_elements(By.TAG_NAME, "li")
    if hEdu:
        edu = hEdu[0].text
        pattern = r',|\('
        edu=re.split(pattern,edu)[0]
    else:
        edu='Not Mentioned'
    f1 = soup1.find('div', class_='profile-info')
    ext=f1.findAll('p')[-1].text
    if('(051)' in ext):
        ext=int(ext.split(': ')[-1])
    else:
        ext='None'
    hec=soup1.find('div',class_='profile-info')
    B_hec="HEC Approved" in (hec.findAll('p')[1].text)
    return ext,edu,B_hec

In [71]:
id_=[]
name=[]
email=[]
desig=[]
img=[]
profile=[]
HEC=[]
Ext=[]
HighestEd=[]
Dept=[]
count=0

In [72]:
def get_data(f_isb,dept):
    temp_c_prof=[]
    for i in range(len(f_isb)):
        faculty = f_isb[i] 
        try:
            id_temp=faculty.find_element(By.CSS_SELECTOR, "ul.social li a")
            id_.append(id_temp.get_attribute("href").split('/')[-1])
        except:
            id_.append(-1)
        faculty = f_isb[i]
        temp_name=faculty.find_element(By.CLASS_NAME, "title").text
        name.append(temp_name)
        desig.append(faculty.find_element(By.CLASS_NAME, "post").text)
        email.append(faculty.find_element(By.TAG_NAME, "p").text)
        temp_img=faculty.find_element(By.TAG_NAME, "img")
        img.append(temp_img.get_attribute("src"))
        temp_url=faculty.find_element(By.CSS_SELECTOR, "ul.social li a")
        temp_url=temp_url.get_attribute("href")
        profile.append(temp_url)
        temp_c_prof.append(temp_url)
    print(len(profile))
    print(len(temp_c_prof))
        
    for i in range(len(temp_c_prof)): 
        Extension,HighestEdu,hec_a=parse_profile_isb(temp_c_prof[i])
        #print(Extension,HighestEdu,dep)
        Ext.append(Extension)
        HighestEd.append(HighestEdu)
        Dept.append(dept)
        HEC.append(hec_a)
    

In [73]:
URL_isb = "http://isb.nu.edu.pk/Faculty/allfaculty/"
driver.get(URL_isb)

driver.implicitly_wait(7)
cs_isb=driver.find_element(By.ID,"facultyCS")
cs_isb=cs_isb.find_elements(By.CLASS_NAME,'our-team')
print(len(cs_isb))
get_data(cs_isb,'CS')

121
121
121


In [74]:
URL_isb = "http://isb.nu.edu.pk/Faculty/allfaculty/"
driver.get(URL_isb)
driver.implicitly_wait(9)
ee_isb=driver.find_element(By.ID,'facultyEE')
ee_isb=ee_isb.find_elements(By.CLASS_NAME,'our-team')
get_data(ee_isb,'EE')

157
36


In [75]:
URL_isb = "http://isb.nu.edu.pk/Faculty/allfaculty/"
driver.get(URL_isb)
driver.implicitly_wait(9)
ms_isb=driver.find_element(By.ID,'facultyMS')
ms_isb=ms_isb.find_elements(By.CLASS_NAME,'our-team')
get_data(ms_isb,'MS')

187
30


In [76]:
URL_isb = "http://isb.nu.edu.pk/Faculty/allfaculty/"
driver.get(URL_isb)
driver.implicitly_wait(9)
sh_isb=driver.find_element(By.ID,'facultySH')
sh_isb=sh_isb.find_elements(By.CLASS_NAME,'our-team')
get_data(sh_isb,'S&H')

204
17


In [77]:
print(len(id_))
print(len(name))
print(len(email))
print(len(desig))
print(len(img))
print(len(profile))
print(len(HEC))
print(len(Ext))
print(len(HighestEd))
print(len(Dept))

204
204
204
204
204
204
204
204
204
204


In [78]:
df2 = pd.DataFrame({'ID':id_,
                  'Name':name,
                  'Designation':desig,
                   'HEC':HEC,
                   'Highest':HighestEd,
                   'Email':email,
                    'Department':Dept,
                   'Extension':Ext,
                   'Campus':'Isb',
                   'Image':img,
                  'Profile':profile
                  })

In [79]:
df2[df2.Name.str.contains('H')]


,ID,Name,Designation,HEC,Highest,Email,Department,Extension,Campus,Image,Profile
2,4400,Hammad Majeed,HoD (Computer Science) & Professor,True,P.hD,hammad.majeed@nu.edu.pk,CS,672,Isb,http://isb.nu.edu.pk/Images/Profile/CS/4400.jpg,http://isb.nu.edu.pk/Faculty/Details/4400
4,4551,Hasan Mujtaba Kiyani,HoD (Artificial Intelligence & Data Science) &...,True,Ph.D,hasan.mujtaba@nu.edu.pk,CS,508,Isb,http://isb.nu.edu.pk/Images/Profile/CS/4551.jpg,http://isb.nu.edu.pk/Faculty/Details/4551
17,5690,Shujaat Hussain Kausar,Associate Professor,False,P.hD,shujaat.hussain@nu.edu.pk,CS,501,Isb,http://isb.nu.edu.pk/Images/Profile/CS/5690.jpg,http://isb.nu.edu.pk/Faculty/Details/5690
22,5200,Usman Habib,Associate Professor,True,PhD,usman.habib@nu.edu.pk,CS,None,Isb,http://isb.nu.edu.pk/Images/Profile/CS/5200.jpg,http://isb.nu.edu.pk/Faculty/Details/5200
30,4381,Jawad Hassan Nisar,Assistant Professor,False,MS,jawad.hassan@nu.edu.pk,CS,630,Isb,http://isb.nu.edu.pk/Images/Profile/CS/4381.jpg,http://isb.nu.edu.pk/Faculty/Details/4381
41,6606,Isma Ul Hassan,Assistant Professor,False,PhD,isma.hassan@nu.edu.pk,CS,699,Isb,http://isb.nu.edu.pk/Images/Profile/CS/6606.jpg,http://isb.nu.edu.pk/Faculty/Details/6606
42,6283,Hassan Sartaj,Assistant Professor,False,Not Mentioned,hassan.sartaj@nu.edu.pk,CS,321,Isb,http://isb.nu.edu.pk/Images/Profile/CS/6283.jpg,http://isb.nu.edu.pk/Faculty/Details/6283
48,6296,Marium Hida,Lecturer,False,Not Mentioned,marium.hida@nu.edu.pk,CS,603,Isb,http://isb.nu.edu.pk/Images/Profile/CS/6296.jpg,http://isb.nu.edu.pk/Faculty/Details/6296
58,6508,Zaheer Ul Hussain Sani,Lecturer,False,MSc,zaheer.sani@nu.edu.pk,CS,None,Isb,http://isb.nu.edu.pk/Images/Profile/CS/6508.jpg,http://isb.nu.edu.pk/Faculty/Details/6508
60,6529,Mehmood Ul Hassan,Lecturer,False,MS,mehmood.hassan@nu.edu.pk,CS,513,Isb,http://isb.nu.edu.pk/Images/Profile/CS/6529.jpg,http://isb.nu.edu.pk/Faculty/Details/6529


In [81]:
df2['Extension'] = df2['Extension'].replace('None',-1)
df2['Extension'] = df2['Extension'].astype(int)
df2.to_csv('Isb_FACULTY.csv')
df2.dtypes

ID             object
Name           object
Designation    object
HEC              bool
Highest        object
Email          object
Department     object
Extension       int32
Campus         object
Image          object
Profile        object
dtype: object

# Karachi Campus

In [64]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import re

In [65]:
def parse_prof_kh(URL):
    r_kh = requests.get(URL) 
    soup_kh = BeautifulSoup(r_kh.content, 'html.parser')
    pattern =r',|\('
    kh=soup_kh.find('span',class_='gdlr-core-title-item-caption').text
    temp_hec='HEC'in kh
    try:
        edu= soup_kh.find(class_='gdlr-core-icon-list-icon fa fa-graduation-cap')
        edu=edu.find_parent('li')
        if edu:
            edu_text = edu.get_text(strip=True)
            edu=re.split(pattern,edu_text)[0].replace(" ","")
        else:
            edu='Not Mentioned'
    except:
        try:
            edu= soup_kh.findAll('span',class_='gdlr-core-title-item-caption')[1].text
            edu=re.split(pattern,edu)[0].replace(" ","")
        except:
            edu='Not Mentioned'
    if 'JournalPapers' in edu:
        edu='Not Mentioned'
    page_id= soup_kh.find('script', {"id":'burst-js-extra'})
    page_id= page_id.text
    pattern = r'"page_id":"(\d+)"'
    match=re.search(pattern,page_id)
    if match:
        page_id = int(match.group(1))
    else:
        page_id=-1
    return edu,temp_hec,page_id

In [66]:
id_=[]
name=[]
email=[]
desig=[]
img=[]
profile=[]
HEC=[]
Ext=[]
HighestEd=[]
Dept=[]

In [67]:
def get_data_kh(f_kh,dept):
    for i in range(len(f_kh)):
        faculty=f_kh[i]
        name.append(faculty.find('h3',class_='gdlr-core-personnel-list-title').text.split(',')[0])
        desig.append((faculty.find('div',class_='gdlr-core-personnel-list-position').text))
        email.append((faculty.find('div',class_='kingster-type-email').text))
        Ext.append(faculty.find('div',class_='kingster-type-phone').text)
        profile.append(faculty.find('a',class_='gdlr-core-personnel-list-button gdlr-core-button')['href'])
        #temp_img=faculty.find('div',class_='gdlr-core-personnel-list-image')
        #img.append(temp_img.find('a')['href'])
        try:
            img_t=faculty.find('div',class_='gdlr-core-personnel-list-image')
            img.append(img_t.find('img')['src'])
        except:
            img.append(-1)
        a,b,c=parse_prof_kh(profile[-1])
        HEC.append(b)
        HighestEd.append(a)
        Dept.append(dept)
        id_.append(c)

In [68]:
URL = "https://khi.nu.edu.pk/faculty-php/"
r = requests.get(URL) 
print(r) 
soup = BeautifulSoup(r.content, 'html.parser')
f =soup.findAll('div',class_='gdlr-core-personnel-list-column')
get_data_kh(f,'CS')

<Response [200]>


In [69]:
URL = "https://khi.nu.edu.pk/department-of-cyber-security/"
r2 = requests.get(URL) 
print(r2) 
soup2= BeautifulSoup(r2.content, 'html.parser')
f2=soup2.findAll('div',class_='gdlr-core-personnel-list-column')
get_data_kh(f2,'CYS')

<Response [200]>


In [70]:
URL = "https://khi.nu.edu.pk/department-of-artificial-intelligence/"
r3= requests.get(URL) 
soup3= BeautifulSoup(r3.content, 'html.parser')
f3=soup3.findAll('div',class_='gdlr-core-personnel-list-column')
get_data_kh(f3,'AI')

In [71]:
URL = "https://khi.nu.edu.pk/department-of-software-engineering/"
r4= requests.get(URL) 
print(r4) 
soup4= BeautifulSoup(r4.content, 'html.parser')
f4=soup4.findAll('div',class_='gdlr-core-personnel-list-column')
get_data_kh(f4,'SE')


<Response [200]>


In [72]:
URL = "https://khi.nu.edu.pk/department-of-electrical-engineering/"
r5= requests.get(URL) 
print(r5) 
soup5= BeautifulSoup(r5.content, 'html.parser')
f5=soup5.findAll('div',class_='gdlr-core-personnel-list-column')
get_data_kh(f5,'EE')

<Response [200]>


In [73]:
URL = "https://khi.nu.edu.pk/department-of-management-sciences/"
r6= requests.get(URL) 
print(r6) 
soup6= BeautifulSoup(r6.content, 'html.parser')
f6=soup6.findAll('div',class_='gdlr-core-personnel-list-column')
get_data_kh(f6,'MS')

<Response [200]>


In [74]:
URL = "https://khi.nu.edu.pk/department-of-sciences-humanities/"
r7= requests.get(URL) 
print(r7) 
soup7= BeautifulSoup(r7.content, 'html.parser')
f7=soup7.findAll('div',class_='gdlr-core-personnel-list-column')
get_data_kh(f7,'S&H')

<Response [200]>


In [75]:
print(len(id_))
print(len(name))
print(len(email))
print(len(desig))
print(len(img))
print(len(profile))
print(len(HEC))
print(len(Ext))
print(len(HighestEd))
print(len(Dept))

133
133
133
133
133
133
133
133
133
133


In [76]:
df3 = pd.DataFrame({'ID':id_,
                  'Name':name,
                  'Designation':desig,
                   'HEC':HEC,
                   'Highest':HighestEd,
                   'Email':email,
                   'Department':Dept,
                   'Extension':Ext,
                   'Campus':'Khi',
                   'Image':img,
                  'Profile':profile
                  })

In [77]:
df3[df3.Name.str.contains('H')]

,ID,Name,Designation,HEC,Highest,Email,Department,Extension,Campus,Image,Profile
1,6455,Dr. Farooque Hassan Kumbhar,Associate Professor and HOD,True,PhDinComputerEngineering,farooque.hassan@nu.edu.pk,CS,161,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/dr-farooque-ha...
8,6532,Dr. Farrukh Hasan Syed,Assistant Professor,True,PhD,farrukh.hasan@nu.edu.pk,CS,132,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/dr-farrukh-has...
14,6575,Mr. Syed Kashan Hussain Naqvi,Assistant Professor,False,ME,kashan.hussain@nu.edu.pk,CS,283,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/mr-shahbaz-sid...
21,7217,Ms. Anaum Hamid,Lecturer,False,MS,anaum.hamid@nu.edu.pk,CS,237,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/ms-anaum-hamid/
26,7244,Mr. Fahad Hussain,Lecturer,False,MS,fahad.hussain@nu.edu.pk,CS,136,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/mr-fahad-hussain/
28,7254,Ms. Hajra Ahmed,Lecturer,False,ME,hajra.ahmed@nu.edu.pk,CS,142,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/ms-hajra-ahmed/
36,7290,Ms. Shahar Bano Husnine,Lecturer,False,MS,shahar.bano@nu.edu.pk,CS,220,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/ms-shahar-bano...
41,7317,Mr. Syed Zain Ul Hassan,Lecturer,False,MS,zain.hassan@nu.edu.pk,CS,197,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/mr-syed-zain-u...
47,7372,Mr. Shakir Hussain,Instructor,False,BS,shakir.hussain@nu.edu.pk,CS,284,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/mr-shakir-huss...
53,6627,Dr. Sufian Hameed,Assistant Professor,False,PhD,sufian.hameed@nu.edu.pk,CYS,297,Khi,https://khi.nu.edu.pk/wp-content/uploads/2023/...,https://khi.nu.edu.pk/personnel/dr-sufian-hameed/


In [78]:
df3['Extension'] = df3['Extension'].replace('-',-1)
df3['Extension'] = df3['Extension'].astype(int)
df3.to_csv('Khi_FACULTY.csv')
df3.dtypes

ID              int64
Name           object
Designation    object
HEC              bool
Highest        object
Email          object
Department     object
Extension       int32
Campus         object
Image          object
Profile        object
dtype: object

In [100]:
nuces_df = pd.concat([df1, df2, df3], ignore_index=True)
nuces_df.to_csv('fast_faculty.csv', index=False)
faculty = pd.read_csv('fast_faculty.csv')
fraction= 4/ 10
sample_= faculty.sample(frac=fraction)
sample_.to_csv('sample.csv', index=False)

# In case not running above complete code

In [84]:
d1=pd.read_csv('Lhr_FACULTY.csv',index_col=0)
d2=pd.read_csv('Isb_FACULTY.csv',index_col=0)
d3=pd.read_csv('Khi_FACULTY.csv',index_col=0)
nuces_df = pd.concat([d1, d2, d3], ignore_index=True)
nuces_df.to_csv('fast_faculty.csv', index=False)
faculty = pd.read_csv('fast_faculty.csv')
fraction= 4/ 10
sample_= faculty.sample(frac=fraction)
sample_.to_csv('sample.csv', index=False)